In [19]:
import locale
import os
from itertools import combinations

import pandas as pd
import tiktoken
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter as utils_get_column_letter
#from openpyxl.worksheet.worksheet import Worksheet

from support_functions.excel_helper import save_to_excel
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.request_handler import process_user_stories, process_user_stories_parallel
from support_functions.data_transformations import remove_pov_and_add_usid
from support_functions.time_recorder import TimeRecorder

In [20]:
locale.setlocale(locale.LC_ALL, 'de_DE') 
MODEL_VERSION_NAME = os.getenv('MODEL_VERSION')
THREADING = bool(int(os.getenv('THREADING')))
print(f"Threading is used: {THREADING}")
print(MODEL_VERSION_NAME)

Threading is used: False
gpt-3.5-turbo


In [21]:
time_recorders: dict = {}
time_recorder: TimeRecorder = None

### Load all User Stories from the different packages

We do not need:
- POS

Highlighting words with #

In [22]:
datasets: dict[str, list] = loading()
print(datasets["g02"][0])

{'PID': '#G02#', 'Text': '#G02# As a Data user, I want to have the 12-19-2017 deletions processed.', 'Persona': ['Data user'], 'Action': {'Primary Action': ['have'], 'Secondary Action': ['processed']}, 'Entity': {'Primary Entity': ['12-19-2017 deletions'], 'Secondary Entity': ['']}, 'Benefit': '', 'Triggers': [['Data user', 'have']], 'Targets': [['processed', '12-19-2017 deletions'], ['have', '12-19-2017 deletions']], 'Contains': [], 'Persona POS': {'Persona POS tag': [['NOUN', 'NOUN']], 'Persona POS text': [['Data', 'user']]}, 'Action POS': {'Primary Action POS': {'Primary Action POS tag': [['VERB']], 'Primary Action POS text': [['have']]}, 'Secondary Action POS': {'Secondary Action POS tag': [['VERB']], 'Secondary Action POS text': [['processed']]}}, 'Entity POS': {'Primary Entity POS': {'Primary Entity POS tag': [['NUM', 'NOUN']], 'Primary Entity POS text': [['12-19-2017', 'deletions']]}, 'Secondary Entity POS': {'Secondary Entity POS tag': [[]], 'Secondary Entity POS text': [[]]}}}

### Keeping the data untouched. Just remove data load.

In [23]:
datasets = remove_pov_and_add_usid(datasets)
datasets["g02"][0]

{'usid': 215,
 'PID': '#G02#',
 'Text': '#G02# As a Data user, I want to have the 12-19-2017 deletions processed.',
 'Persona': ['Data user'],
 'Action': {'Primary Action': ['have'], 'Secondary Action': ['processed']},
 'Entity': {'Primary Entity': ['12-19-2017 deletions'],
  'Secondary Entity': ['']},
 'Benefit': '',
 'Triggers': [['Data user', 'have']],
 'Targets': [['processed', '12-19-2017 deletions'],
  ['have', '12-19-2017 deletions']],
 'Contains': [],
 'Persona POS': {'Persona POS tag': [['NOUN', 'NOUN']],
  'Persona POS text': [['Data', 'user']]},
 'Action POS': {'Primary Action POS': {'Primary Action POS tag': [['VERB']],
   'Primary Action POS text': [['have']]},
  'Secondary Action POS': {'Secondary Action POS tag': [['VERB']],
   'Secondary Action POS text': [['processed']]}},
 'Entity POS': {'Primary Entity POS': {'Primary Entity POS tag': [['NUM',
     'NOUN']],
   'Primary Entity POS text': [['12-19-2017', 'deletions']]},
  'Secondary Entity POS': {'Secondary Entity POS

### Prepaire Prompting

### Request definition to ChatGPT

In [24]:
def transform_2_dataframe(data: list) -> pd.DataFrame:
    rows = []
    for entry in data:
        row = {
            "PID": entry["PID"],
            "USID": entry["usid"],
            "User Story Text": entry["Text"],
            "Main Part": entry["Main Part"],
            "Benefit": entry["Benefit"],
            "Triggers Main Part": entry["Triggers"]["Main Part"],
            "Triggers Benefit": entry["Triggers"]["Benefit"],
            "Targets Main Part": entry["Targets"]["Main Part"],
            "Targets Benefit": entry["Targets"]["Benefit"],
            "Contains Main Part": entry["Contains"]["Main Part"],
            "Contains Benefit": entry["Contains"]["Benefit"]
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [25]:
def transform_pairwise(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for i, j in combinations(range(df.shape[0]), 2):
        row = {
            "First PID": df.iloc[i, 0],
            "First USID": df.iloc[i, 1],
            "First User Story Text": df.iloc[i, 2],
            "First Main Part": df.iloc[i, 3],
            "First Benefit": df.iloc[i, 4],
            "First Triggers Main Part": df.iloc[i, 5],
            "First Triggers Benefit": df.iloc[i, 6],
            "First Targets Main Part": df.iloc[i, 7],
            "First Targets Benefit": df.iloc[i, 8],
            "First Contains Main Part": df.iloc[i, 9],
            "First Contains Benefit": df.iloc[i, 10],
            "Second PID": df.iloc[j, 0],
            "Second USID": df.iloc[j, 1],
            "Second User Story Text": df.iloc[j, 2],
            "Second Main Part": df.iloc[j, 3],
            "Second Benefit": df.iloc[j, 4],
            "Second Triggers Main Part": df.iloc[j, 5],
            "Second Triggers Benefit": df.iloc[j, 6],
            "Second Targets Main Part": df.iloc[j, 7],
            "Second Targets Benefit": df.iloc[j, 8],
            "Second Contains Main Part": df.iloc[j, 9],
            "Second Contains Benefit": df.iloc[j, 10]
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [26]:
### Convert the data back
### Add the Type Hinting
### Use this function in processing the pairs
### Processing the pairs has to be done more abstract to use it
def convert_dataframe_to_json(pairs):
    pass

## Data processing for: G02 federal funding

In [27]:
ID_G02: str = "g02"
data_g02: list = datasets[ID_G02]
df_g02: pd.DataFrame = transform_2_dataframe(data_g02)
df_g02_pairs = transform_pairwise(df_g02)

KeyError: 'Main Part'

In [ ]:
display(df_g02)

,PID,User Story Text,Main Part,Benefit,Triggers Main Part,Triggers Benefit,Targets Main Part,Targets Benefit,Contains Main Part,Contains Benefit
0,#G02#,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"[[Data user, have]]",[],"[[have, 12-19-2017 deletions]]",[],[],[]
1,#G02#,"As a UI designer, I want to redesign the Resou...","As a UI designer, I want to redesign the Resou...",it matches the new Broker design styles,"[[UI designer, redesign]]",[],"[[redesign, Resources page]]","[[matches, new Broker design styles]]",[],[]
2,#G02#,"As a UI designer, I want to report to the Agen...","As a UI designer, I want to report to the Agen...",they are aware of their contributions to makin...,"[[UI designer, report]]",[],"[[report, user testing]]","[[aware, contributions], [making, Broker]]",[],[]
3,#G02#,"As a UI designer, I want to move on to round 2...","As a UI designer, I want to move on to round 2...",I can get approvals from leadership,"[[UI designer, move on]]",[],"[[move on, round 2]]","[[get, approvals]]",[],[]
4,#G02#,"As a UI designer, I want to move on to round 3...","As a UI designer, I want to move on to round 3...",I can get approvals from leadership,"[[UI designer, move on]]",[],"[[move on, round 3]]","[[get, approvals]]",[],[]
...,...,...,...,...,...,...,...,...,...,...
90,#G02#,"As a FABS user, I want to have my validations ...","As a FABS user, I want to have my validations ...",,"[[FABS user, have]]",[],"[[have, validations]]","[[run, reasonable amount of time]]",[],[]
91,#G02#,"As a FABS user, I want to see correct status l...","As a FABS user, I want to see correct status l...",I can quickly see my submission history,"[[FABS user, see]]",[],"[[see, correct status labels]]","[[quickly see, submission history]]",[],[]
92,#G02#,"As an agency user, I want to know when the sub...","As an agency user, I want to know when the sub...",I know when the submission starts and ends,"[[agency user, know]]",[],"[[know, submission periods start and end]]","[[know, submission starts and ends]]",[],[]
93,#G02#,"As an agency user, I want a landing page to na...","As an agency user, I want a landing page to na...",I can access both sides of the site,"[[agency user, want]]",[],"[[want, landing page]]","[[navigate, FABS], [navigate, DABS pages], [ac...",[],[]


In [ ]:
display(df_g02_pairs)

,First PID,First User Story Text,First Main Part,First Benefit,First Triggers Main Part,First Triggers Benefit,First Targets Main Part,First Targets Benefit,First Contains Main Part,First Contains Benefit,Second PID,Second User Story Text,Second Main Part,Second Benefit,Second Triggers Main Part,Second Triggers Benefit,Second Targets Main Part,Second Targets Benefit,Second Contains Main Part,Second Contains Benefit
0,#G02#,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"[[Data user, have]]",[],"[[have, 12-19-2017 deletions]]",[],[],[],#G02#,"As a UI designer, I want to redesign the Resou...","As a UI designer, I want to redesign the Resou...",it matches the new Broker design styles,"[[UI designer, redesign]]",[],"[[redesign, Resources page]]","[[matches, new Broker design styles]]",[],[]
1,#G02#,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"[[Data user, have]]",[],"[[have, 12-19-2017 deletions]]",[],[],[],#G02#,"As a UI designer, I want to report to the Agen...","As a UI designer, I want to report to the Agen...",they are aware of their contributions to makin...,"[[UI designer, report]]",[],"[[report, user testing]]","[[aware, contributions], [making, Broker]]",[],[]
2,#G02#,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"[[Data user, have]]",[],"[[have, 12-19-2017 deletions]]",[],[],[],#G02#,"As a UI designer, I want to move on to round 2...","As a UI designer, I want to move on to round 2...",I can get approvals from leadership,"[[UI designer, move on]]",[],"[[move on, round 2]]","[[get, approvals]]",[],[]
3,#G02#,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"[[Data user, have]]",[],"[[have, 12-19-2017 deletions]]",[],[],[],#G02#,"As a UI designer, I want to move on to round 3...","As a UI designer, I want to move on to round 3...",I can get approvals from leadership,"[[UI designer, move on]]",[],"[[move on, round 3]]","[[get, approvals]]",[],[]
4,#G02#,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"[[Data user, have]]",[],"[[have, 12-19-2017 deletions]]",[],[],[],#G02#,"As a Developer , I want to be able to log bett...","As a Developer , I want to be able to log better",I can troubleshoot issues with particular subm...,"[[Developer, log better]]","[[functions, issues], [particular submissions,...",[],"[[troubleshoot, issues]]",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,#G02#,"As a FABS user, I want to see correct status l...","As a FABS user, I want to see correct status l...",I can quickly see my submission history,"[[FABS user, see]]",[],"[[see, correct status labels]]","[[quickly see, submission history]]",[],[],#G02#,"As an agency user, I want a landing page to na...","As an agency user, I want a landing page to na...",I can access both sides of the site,"[[agency user, want]]",[],"[[want, landing page]]","[[navigate, FABS], [navigate, DABS pages], [ac...",[],[]
4461,#G02#,"As a FABS user, I want to see correct status l...","As a FABS user, I want to see correct status l...",I can quickly see my submission history,"[[FABS user, see]]",[],"[[see, correct status labels]]","[[quickly see, submission history]]",[],[],#G02#,"As an agency user, I want to submit my data el...","As an agency user, I want to submit my data el...",Excel won't strip off leading and trailing zeroes,"[[agency user, submit]]",[],"[[submit, data elements surrounded by quotatio...",[],[],[]
4462,#G02#,"As an agency user, I want to know when the sub...","As an agency user, I want to know when the sub...",I know when the submission starts and ends,"[[agency user, know]]",[],"[[know, submission periods start and end]]","[[know, submission starts and ends]]",[],[],#G02#,"As an agency user, I want a landing page to n

### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G02] = time_recorder
if THREADING:
    process_user_stories_parallel(message, df_g02_pairs, ID_G02, MODEL_VERSION_NAME, time_recorder=time_recorder)
    print(time_recorder.milliseconds)
else:
    process_user_stories(message, df_g02_pairs, ID_G02, MODEL_VERSION_NAME, time_recorder=time_recorder)

### Process and Store the User Stories persistently

## Save total Speeds

In [ ]:
SHEET_NAME = 'Time Consup. Anlys.S.'

In [ ]:
base_path: str = os.getcwd()
path_to_file = os.path.join(base_path, "results")
path_to_file = os.path.join(path_to_file, "redundancy-model-" + MODEL_VERSION_NAME)

count_runs_per_data_set: dict[str, int] = {}
for key in time_recorders.keys():
        idx = 0
        _ =  f"{idx:02d}_{key}.json"
        while os.path.exists(os.path.join(path_to_file, f"{idx:02d}_{key}.json")):
                idx += 1
        count_runs_per_data_set[key] = idx

entries: list[tuple[int,str,str, int, float, float, float]] = []

entry: tuple[int,str,str, int, float, float, float] = None
for key, value in time_recorders.items():
        entry = (key, count_runs_per_data_set[key], MODEL_VERSION_NAME, str(THREADING).lower(), 
                 value.nanoseconds, value.milliseconds, value.seconds, value.minutes)
        entries.append(entry)

columns = ['Dataset', 'Run Count', 'Model Version', 'Threading Enabled', 
           'Nanoseconds', 'Milliseconds', 'Seconds', 'Minutes']

_file_path: str = os.path.join(base_path, os.getenv('OUTPUT_EXCEL_NAME_WITHOUT_ANNOTATIONS'))
check = os.path.exists(_file_path)

old_time_consuption_data: pd.DataFrame = None
if check:
        try:
                old_time_consuption_data = pd.read_excel(_file_path, SHEET_NAME)
        except ValueError:
                check = False
           
time_consumption_data: pd.DataFrame = pd.DataFrame(entries, columns=columns)

if check and not old_time_consuption_data.empty:
        # Checking if in the excel is already the data. Case: this code is executed twice for the same data
        old_time_consuption_data = old_time_consuption_data.dropna()
        for idx in range(len(old_time_consuption_data)):
                condition = time_consumption_data[
                        (time_consumption_data['Dataset'] == old_time_consuption_data.iat[idx, 0]) & 
                        (time_consumption_data['Run Count'] == old_time_consuption_data.iat[idx, 1])
                ].index
                time_consumption_data = time_consumption_data.drop(condition).reset_index(drop=True)
        time_consumption_data = pd.concat([old_time_consuption_data, time_consumption_data]).reset_index(drop=True)

In [ ]:
def formatter_time(wb: Workbook, sheet_name: str): 
    ws = wb[sheet_name]
    header_font = Font(size=14, bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
    
    ADDITIONAL_LENGTH: int = 0
    ADJUSTED_WIDTH: int = 0
    MAX_LEN: int = 0
    for col in ws.iter_cols(min_row=1, max_row=1):
        for cell in col:
            MAX_LEN = len(str(cell.value))
            ADDITIONAL_LENGTH = (MAX_LEN + 2)
            ADJUSTED_WIDTH = 0
            ADJUSTED_WIDTH =  ADDITIONAL_LENGTH * 1.5
            ws.column_dimensions[utils_get_column_letter(cell.column)].width = ADJUSTED_WIDTH
            
    alignment = Alignment(vertical='center', horizontal='left')
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = alignment

    num_columns = ws.max_column
    header_range = f"A1:{utils_get_column_letter(num_columns)}1"
    ws.auto_filter.ref = header_range
    ws.freeze_panes = ws['A2']

    wrap_alignment = Alignment(wrap_text=True, vertical='top', horizontal='left')
    for row in ws.iter_rows(min_row=2):
        for cell in row:
            cell.alignment = wrap_alignment

In [ ]:
save_to_excel(time_consumption_data, formatter_time, SHEET_NAME, os.getenv('OUTPUT_EXCEL_NAME_WITHOUT_ANNOTATIONS'))